# Parent Child

Trying an idea... Each node in the product potentially has a parent. If the node does not have a parent, it is the root node or `product`. If it has a `parent` and `children` , it is an `assembly`. If it has no `children`, it is a part.

If it does not have a `parent`, it is on the top line of the contruction tree.
If it has a `parent` and `children`, it is in the middle somewhere.
If it has only a `parent`, it is the baseline item in the contruction tree.

## Questions:

- can a node have multiple labels? Assume yes
- can we run an `on-update` type trigger in neo4j to set the `product`, `assembly`, and `part` labels?

In [53]:
# Dependencies and imports
!pip install lorem-text

from lorem_text import lorem
from yfiles_jupyter_graphs import GraphWidget

In [100]:
# Connect to DB
from neo4j import GraphDatabase

# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
URI = "neo4j+s://89698250.databases.neo4j.io"
AUTH = ("neo4j", "nN4v9Y33RjQtEPzqgNAUUbKQj6Os9Hbs_AuGSY7s1DQ")

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

In [108]:
# Clear the DB - uncomment to use
records, summary, keys = driver.execute_query(
    "MATCH (n) DETACH DELETE (n);",
    database_="neo4j",
)

In [109]:
# Create the Suppliers

suppliers = [
    {'code': 'FR001', 'name': 'XYZ Co (self)', 'country': "FR"},
    {'code': 'FR002', 'name': 'Supplier #1', 'country': "FR"},
    {'code': 'CN001', 'name': 'Supplier #2', 'country': "CN"},
    {'code': 'US001', 'name': 'Supplier #3', 'country': "US"},
    {'code': 'UK001', 'name': 'Supplier #4', 'country': "UK"},
]

cypher_query = """
WITH $json as data
UNWIND data AS s 
MERGE (supplier:Supplier {code:s.code}) SET
	supplier.name = s.name, 
	supplier.country = s.country
"""

with driver.session() as session:
    result = session.run(cypher_query, json = suppliers)


print("Done")


Done


In [110]:
# items JSON

items = [
    {
        "sku": "ss01",
        "name": "SuperSport",
        "retail_price": 29999,
        "qty": 1,
        "order": 0,
        "type": "assembled"
    },
    {
        "sku": "co01",
        "name": "Color",
        "parent": "ss01",
        "qty": 2,
        "order": 1,
        "type": "assembled"
    },
    {
        "sku": "drk_grn",
        "name": "Dark Green",
        "qty": 2,
        "order": 1,
        "parent": "co01",
        "type": "purchased"
    },
    {
        "sku": "whl01",
        "name": "Wheel",
        "qty": 2,
        "order": 1,
        "parent": "ss01",
        "type": "assembled"
    },
    {
        "sku": "sp01",
        "name": "Spoke - STD",
        "uom": "pc",
        "qty": 21,
        "order": 2,
        "parent": "whl01",
        "suppliers":
        [
            {
                "code": "CN001",
                "cost": 100
            },
            {
                "code": "FR002",
                "cost": 110
            },
            {
                "code": "US001",
                "cost": 120
            }
        ],
        "selected_supplier":
        {
            "code": "CN001",
            "cost": 100
        },
        "type": "purchased"
    },
    {
        "sku": "hb01",
        "name": "Hub",
        "uom": "pc",
        "qty": 1,
        "order": 1,
        "parent": "whl01",
        "suppliers":
        [
            {
                "code": "CN001",
                "cost": 60
            },
            {
                "code": "US001",
                "cost": 89
            }
        ],
        "selected_supplier":
        {
            "code": "CN001",
            "cost": 60
        },
        "type": "purchased"
    },
    {
        "sku": "rm01",
        "name": "Rim",
        "uom": "pc",
        "qty": 1,
        "order": 3,
        "parent": "whl01",
        "suppliers":
        [
            {
                "code": "CN001",
                "cost": 600
            },
            {
                "code": "US001",
                "cost": 510
            }
        ],
        "selected_supplier":
        {
            "code": "CN001",
            "cost": 600
        },
        "type": "purchased"
    },
    {
        "sku": "ty01",
        "name": "Tyre - STD",
        "uom": "pc",
        "qty": 1,
        "order": 5,
        "parent": "whl01",
        "suppliers":
        [
            {
                "code": "FR002",
                "cost": 568
            },
            {
                "code": "CN001",
                "cost": 320
            },
            {
                "code": "US001",
                "cost": 700
            }
        ],
        "selected_supplier":
        {
            "code": "FR002",
            "cost": 568
        },
        "type": "purchased"
    },
    {
        "sku": "vlv01",
        "name": "Valve",
        "uom": "pc",
        "qty": 1,
        "order": 4,
        "parent": "whl01",
        "suppliers":
        [
            {
                "code": "FR002",
                "cost": 42
            },
            {
                "code": "CN001",
                "cost": 25
            },
            {
                "code": "US001",
                "cost": 32
            }
        ],
        "selected_supplier":
        {
            "code": "FR002",
            "cost": 42
        },
        "type": "purchased"
    },
    {
        "sku": "fs01",
        "name": "Front Set",
        "uom": "pc",
        "qty": 1,
        "order": 2,
        "parent": "ss01",
        "type": "assembled"
    },
    {
        "sku": "hbr01",
        "name": "Handlebar",
        "uom": "pc",
        "qty": 1,
        "order": 1,
        "parent": "fs01",
        "suppliers":
        [
            {
                "code": "FR002",
                "cost": 234
            }
        ],
        "selected_supplier":
        {
            "code": "FR002",
            "cost": 234
        },
        "type": "purchased"
    },
    {
        "sku": "hdg01",
        "name": "Grip",
        "uom": "pc",
        "qty": 2,
        "order": 2,
        "parent": "fs01",
        "suppliers":
        [
            {
                "code": "CN001",
                "cost": 121
            },
            {
                "code": "FR002",
                "cost": 234
            },
            {
                "code": "US001",
                "cost": 210
            }
        ],
        "selected_supplier":
        {
            "code": "CN001",
            "cost": 121
        },
        "type": "purchased"
    },
    {
        "sku": "fb01",
        "name": "Front brake",
        "uom": "pc",
        "qty": 1,
        "order": 3,
        "parent": "fs01",
        "suppliers":
        [
            {
                "code": "CN001",
                "cost": 451
            },
            {
                "code": "US001",
                "cost": 612
            }
        ],
        "selected_supplier":
        {
            "code": "CN001",
            "cost": 451
        },
        "type": "purchased"
    },
    {
        "sku": "fo01",
        "name": "Fork",
        "uom": "pc",
        "qty": 1,
        "order": 5,
        "parent": "fs01",
        "suppliers":
        [
            {
                "code": "CN001",
                "cost": 213
            },
            {
                "code": "FR001",
                "cost": 250
            },
            {
                "code": "FR002",
                "cost": 300
            },
            {
                "code": "US001",
                "cost": 320
            }
        ],
        "selected_supplier":
        {
            "code": "CN001",
            "cost": 213
        },
        "type": "purchased"
    },
    {
        "sku": "sh01",
        "name": "Shifter",
        "uom": "pc",
        "qty": 1,
        "order": 4,
        "parent": "fs01",
        "suppliers":
        [
            {
                "code": "CN001",
                "cost": 221
            },
            {
                "code": "US001",
                "cost": 240
            }
        ],
        "selected_supplier":
        {
            "code": "FR002",
            "cost": 221
        },
        "type": "purchased"
    },
    {
        "sku": "sa01",
        "name": "Saddle Area",
        "uom": "pc",
        "qty": 1,
        "order": 3,
        "parent": "ss01",
        "type": "assembled"
    },
    {
        "sku": "s01",
        "name": "Saddle",
        "qty": 1,
        "order": 2,
        "uom": "pc",
        "parent": "sa01",
        "suppliers":
        [
            {
                "code": "CN001",
                "cost": 111
            },
            {
                "code": "UK001",
                "cost": 125
            }
        ],
        "selected_supplier":
        {
            "code": "CN001",
            "cost": 111
        },
        "type": "purchased"
    },
    {
        "sku": "se02",
        "name": "Seat post",
        "qty": 1,
        "uom": "pc",
        "order": 1,
        "parent": "sa01",
        "suppliers":
        [
            {
                "code": "CN001",
                "cost": 222
            },
            {
                "code": "US001",
                "cost": 300
            }
        ],
        "selected_supplier":
        {
            "code": "CN001",
            "cost": 222
        },
        "type": "purchased"
    },
    {
        "sku": "tr01",
        "name": "Transmission",
        "uom": "pc",
        "qty": 1,
        "order": 5,
        "parent": "ss01",
        "type": "assembled"
    },
    {
        "sku": "fd01",
        "name": "Front Derailleur",
        "qty": 1,
        "order": 2,
        "parent": "tr01",
        "suppliers":
        [
            {
                "code": "US001",
                "cost": 451
            }
        ],
        "selected_supplier":
        {
            "code": "US001",
            "cost": 451
        },
        "type": "purchased"
    },
    {
        "sku": "ch01",
        "name": "Chain",
        "qty": 1,
        "order": 1,
        "parent": "tr01",
        "suppliers":
        [
            {
                "code": "CN001",
                "cost": 234
            }
        ],
        "selected_supplier":
        {
            "code": "CN001",
            "cost": 234
        },
        "type": "purchased"
    },
    {
        "sku": "cr01",
        "name": "Chain Rings",
        "qty": 1,
        "order": 3,
        "parent": "tr01",
        "suppliers":
        [
            {
                "code": "CN001",
                "cost": 60
            },
            {
                "code": "FR002",
                "cost": 60
            },
            {
                "code": "US001",
                "cost": 89
            }
        ],
        "selected_supplier":
        {
            "code": "CN001",
            "cost": 60
        },
        "type": "purchased"
    },
    {
        "sku": "rb01",
        "name": "Rear brakes",
        "qty": 1,
        "order": 4,
        "parent": "tr01",
        "suppliers":
        [
            {
                "code": "CN001",
                "cost": 60
            },
            {
                "code": "US001",
                "cost": 89
            }
        ],
        "selected_supplier":
        {
            "code": "CN001",
            "cost": 60
        },
        "type": "purchased"
    },
    {
        "sku": "cs01",
        "name": "Cog set",
        "qty": 1,
        "order": 5,
        "parent": "tr01",
        "suppliers":
        [
            {
                "code": "CN001",
                "cost": 261
            },
            {
                "code": "US001",
                "cost": 211
            }
        ],
        "selected_supplier":
        {
            "code": "CN001",
            "cost": 261
        },
        "type": "purchased"
    },
    {
        "sku": "rd01",
        "name": "Rear Derailluer",
        "qty": 1,
        "order": 6,
        "parent": "tr01",
        "suppliers":
        [
            {
                "code": "CN001",
                "cost": 340
            },
            {
                "code": "US001",
                "cost": 432
            }
        ],
        "selected_supplier":
        {
            "code": "CN001",
            "cost": 340
        },
        "type": "purchased"
    },
    {
        "sku": "pd01",
        "name": "Pedal",
        "qty": 2,
        "order": 8,
        "parent": "tr01",
        "suppliers":
        [
            {
                "code": "CN001",
                "cost": 60
            },
            {
                "code": "FR002",
                "cost": 80
            },
            {
                "code": "US001",
                "cost": 62
            }
        ],
        "selected_supplier":
        {
            "code": "CN001",
            "cost": 60
        },
        "type": "purchased"
    },
    {
        "sku": "ca01",
        "name": "Crank Arm",
        "qty": 2,
        "order": 9,
        "parent": "tr01",
        "suppliers":
        [
            {
                "code": "CN001",
                "cost": 60
            },
            {
                "code": "FR001",
                "cost": 120
            },
            {
                "code": "US001",
                "cost": 89
            }
        ],
        "selected_supplier":
        {
            "code": "CN001",
            "cost": 60
        },
        "type": "purchased"
    },
    {
        "sku": "fr01",
        "name": "Frame",
        "qty": 1,
        "order": 4,
        "parent": "ss01",
        "type": "assembled"
    },
    {
        "sku": "fr02",
        "name": "Frame",
        "uom": "pc",
        "qty": 1,
        "order": 1,
        "parent": "fr01",
        "suppliers":
        [
            {
                "code": "CN001",
                "cost": 212
            },
            {
                "code": "FR001",
                "cost": 250
            },
            {
                "code": "FR002",
                "cost": 300
            },
            {
                "code": "US001",
                "cost": 320
            }
        ],
        "selected_supplier":
        {
            "code": "CN001",
            "cost": 212
        },
        "type": "purchased"
    },
    {
        "sku": "st01",
        "name": "Stem",
        "uom": "pc",
        "qty": 1,
        "order": 2,
        "parent": "fr01",
        "suppliers":
        [
            {
                "code": "FR001",
                "cost": 400
            }
        ],
        "selected_supplier":
        {
            "code": "FR001",
            "cost": 400
        },
        "type": "manufactured"
    },
    {
        "sku": "hd01",
        "name": "Head",
        "uom": "pc",
        "qty": 1,
        "order": 3,
        "parent": "fr01",
        "suppliers":
        [
            {
                "code": "CN001",
                "cost": 500
            },
            {
                "code": "US001",
                "cost": 550
            }
        ],
        "selected_supplier":
        {
            "code": "CN001",
            "cost": 500
        },
        "type": "purchased"
    }
]

In [111]:
# Insert Items

cypher_query = """
WITH $json as data
UNWIND data AS i 
MERGE (item:Item {sku:i.sku}) SET
	item.name = i.name, 
	item.retail_price = i.retail_price, 
	item.uom = i.uom, 
	item.qty = i.qty, 
	item.order = i.order,
    item.type = i.type

FOREACH (s IN i.suppliers | 
	// Find the supplier
	MERGE (supplier:Supplier {code: s.code}) 
    
    // Create the relationship
    MERGE (item)-[:IS_SOURCED_FROM {cost: s.cost}]->(supplier)
)

FOREACH(ignoreMe IN CASE WHEN i.selected_supplier IS NOT NULL THEN [1] ELSE [] END | 
	// Find the supplier
	MERGE (supplier:Supplier {code: i.selected_supplier.code}) 
    
    // Create the relationship
	MERGE (item)-[:SELECTED {cost: i.selected_supplier.cost}]->(supplier)
)

FOREACH(ignoreMe IN CASE WHEN i.parent IS NOT NULL THEN [1] ELSE [] END | 
	// Find the parent item
	MERGE (parent:Item {sku: i.parent}) 

    // Set relationships
    MERGE (parent)-[:PARENT]->(item)
	// MERGE (item)-[:CHILD]->(parent)
)

"""

with driver.session() as session:
    result = session.run(cypher_query, json = items)

print("Done")



Done


In [112]:
# Query DB for Items

with driver.session(database="neo4j") as session:
  graph = session.run("MATCH p=(Item)-[]->(:Item) RETURN p").graph()

# graph


In [113]:
# custom graph functions

def custom_node_color_mapping(index: int, node: dict):
    # print(node)
    if node["properties"]["label"] == "Item":
        return "green"
    elif node["properties"]["label"] == "Product":
        return "red"
    elif node["properties"]["label"] == "Assembly":
        return "orange"
    elif node["properties"]["label"] == "Specification":
        return "blue"
    elif node["properties"]["label"] == "Description":
        return "cyan"
    elif node["properties"]["label"] == "Supplier":
        return "pink"
    
def custom_node_label_mapping(index: int, node: dict):
    # print(node)
    if node["properties"]["label"] == "Item":
        return node["properties"]["sku"] + " - " + node["properties"]["name"]
    elif node["properties"]["label"] == "Product":
        return node["properties"]["sku"] + " - " + node["properties"]["name"]
    elif node["properties"]["label"] == "Assembly":
        return node["properties"]["sku"] + " - " + node["properties"]["name"]
    elif node["properties"]["label"] == "Specification":
        return "v: " + node["properties"]["version"]
    elif node["properties"]["label"] == "Description":
        return "v: " + node["properties"]["version"]
    elif node["properties"]["label"] == "Supplier":
        return node["properties"]["name"] + " - " + node["properties"]["country"]


# TODO

 - set weight of node based attribute, eg cost [Problem: cost lives on the relationship, not the node]

In [115]:
# Render ALL graph...

w = GraphWidget(graph=graph)
w.directed = False
w.set_graph_layout("hierarchic")
# w.set_neighborhood(1,[graph.nodes.get(0).element_id])
w.set_sidebar(enabled = False, start_with = "Neighbourhood")
w.set_overview(False)

w.set_node_color_mapping(custom_node_color_mapping)
# w.set_node_color_mapping(lambda index, node : "blue" if node["properties"]["label"] == "Part" else ("orange" if node["properties"]["label"] == "SubAssembly" else "red"))
w.set_edge_color_mapping(lambda index, edge : "orange")
# w.set_node_label_mapping(lambda index, node : node["properties"]["sku"] + " - " + node["properties"]["name"])
w.set_node_label_mapping(custom_node_label_mapping)
# w.set_node_scale_factor_mapping(lambda index,node: node["properties"]["cost"] / 1000 if node["properties"]["label"] == "Part" else 1)

w.show()


GraphWidget(layout=Layout(height='500px', width='100%'))